#  Syft Duet - Alice 

In [27]:
# the length of iris dataset is 150
# how many samples would you let Alice hold?
data_owner = "Alice"
data_size = 70

In [28]:
#check data
if data_size <= 0:
    data_size=1
if data_size > 150:
    data_size=150

## PART 1: Launch a Duet Server and Connect

As a Data Owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server within this notebook. We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a Data Scientist) connect to this server.

You do this by running the code below and sending the code snippet containing your unique Server ID to your partner and following the instructions it gives!

--from openmind course

In [29]:
#launch the server
import syft as sy
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/home/fjunyuan/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/fjunyuan/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore


If you are in Jupyter Notebook (not Colab) the ☝🏾DUET LIVE STATUS above will animate 

# Part 2: Upload data to Duet Server

Let's say the data owner has a dataset of Iris flowers. He will upload the data to the duet server for other data scientists to use.

In [30]:
from sklearn import datasets
import torch as th

In [31]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

In [32]:
print(type(X))

<class 'numpy.ndarray'>


In [33]:
#adjust size
if data_owner == "Alice":
    X = X[0:data_size]#Bob X  = X[(150-data_size):]
    y = y[0:data_size]#Bob y  = y[(150-data_size):]
if data_owner == "Bob":
    X  = X[(150-data_size):]
    y  = y[(150-data_size):]

For doing machine learning using torch, we need the data to be converted to FloatTensors. Here, the data owner is explicitly doing the conversion before uploading the data. If he doesn't do that, it has to be converted in the data scientist's end as you've seen in the previous exercise.

In [34]:
X = th.FloatTensor(X)
y = th.FloatTensor(y)

In [35]:
print(type(X))

<class 'torch.Tensor'>


In [36]:
#add tag and describtion
X = X.tag("Alice_iris-data")
y = y.tag("Alice_iris-target")

X = X.describe(
    f"This is a dataset for flower classification of {data_size} samples. 4 Features are sepal length (cm)," 
    "sepal width (cm), petal length (cm), petal width (cm)"
)
y = y.describe("Labels for flowers: Iris-setosa, Iris-versicolour, Iris-virginica")

In [37]:
#send pointable dataset to server, data scientist can access to the dataset only if (s)he make a request
data_pointer = X.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)

In [38]:
# Once uploaded, the data owner can see the object stored in the tensor
duet.store

[<syft.proxy.torch.TensorPointer object at 0x7f7d06bef880>, <syft.proxy.torch.TensorPointer object at 0x7f7d06befa90>]

In [39]:
# To see it in a human-readable format, data owner can also pretty-print the tensor information
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: f909fc476ad04dfd8ae1d77ced041b08>,[Alice_iris-data],This is a dataset for flower classification of...,<class 'torch.Tensor'>
1,<UID: 9a3f2a91789e406aa0cac295105e9387>,[Alice_iris-target],"Labels for flowers: Iris-setosa, Iris-versicol...",<class 'torch.Tensor'>


## PART 3: Add Request Handlers

The Data Science Notebook makes a number of requests to access data. You can view these requests with:  
```
duet.requests.pandas
```
Then manually accept or deny them with:
```
duet.requests[0].accept()
```
However for training loops, this can be slow and tedious so the below code will create request handlers which will automatically respond with `accept` or `deny` depending on the name of the request.

In [40]:
duet.requests.pandas

""


In [41]:
# currently we do not have any protection from the handler, handler passes any requests
duet.requests.add_handler(
    action="accept",
    print_local=True  # print the result in your notebook
)

[2022-07-10T15:40:06.259505+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: dcc3b4b4425b4707acc3288b2f6f4b10>: To evaluate training progress
Value: <Storable: 1.1967906951904297>
[2022-07-10T15:40:07.382237+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 1a17c42a627d46538d01fd01caad90d9>: To evaluate training progress
Value: <Storable: 0.630227267742157>
[2022-07-10T15:40:08.495039+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 93875b7093624dd38044f9b7161f1b66>: To evaluate training progress
Value: <Storable: 0.9017127752304077>
[2022-07-10T15:40:09.612604+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: e949a5afb55f4ea6a6c0a7e7617e4c92>: To evaluate training progress
Value: <Storable: 0.5787566900253296>
[2022-07-10T15:40:10.725571+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 398c6bb278464b78a8e860b5da00bd4a>: To evaluate training progress
Value: <Storable: 0.5602102875709534>
[2022-07-10T15:40:11.843198+0300][CRITICAL][logger]][849018] 

[2022-07-10T15:47:27.037644+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 63ae80006f414bad8d79ed8764398618>:
Value: <Storable: 'in_features=4, out_features=20, bias=True'>
[2022-07-10T15:47:27.169897+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 1a75fb57fc354c7daa9433f18f232781>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.0178, -0.5114, -0.9144, -0.3233],    [-0.1618, -0.4166, -1.0175, -0.4378],    [-0.3976, -0.1940, -0.2478, -0.5188],    [-0.1380, -0.4091, 0.0014, 0.4134],    [ 0.0227, -0.0133, -0.2915, -1.1410],    [-0.1855, 0.0480, -0.4581, -0.2016],    [-0.0667, -0.7287, -0.0245, -0.1607],    [-0.2686, -0.3616, -0.3187, 0.0046],    [-0.4551, 0.3928, 0.3300, -0.1806],    [ 0.1730, 0.7135, -0.4616, -1.0475],    [ 0.2288, 0.1352, -0.8260, -0.7280],    [-0.1607, -0.1656, -0.7050, -0.2117],    [-0.2105, -0.2670, -0.2767, -0.1688],    [-0.0371, 0.0421, -0.3847, 0.3509],    [-0.3992, -0.1279, -0.5426, -0.7000],    [-0.4986, -0.6482, -0.0982, -0.4327],    [

[2022-07-10T15:47:30.487891+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 248fbf879b9f4192ba86fb8fef5084ad>:
Value: <Storable: 'in_features=30, out_features=30, bias=True'>
[2022-07-10T15:47:31.609889+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: c68b778c3618404092609e61abd06fcd>:
Value: <Storable: OrderedDict([('weight', tensor([[ 2.9269e-01, -1.6785e-01, 4.5387e-01, 2.6658e-01, -3.5345e-02,     5.1731e-01, 5.3217e-01, 5.3964e-01, 4.4317e-01, 5.0218e-01,     -1.6939e-02, 5.1608e-01, -2.6000e-01, -3.0642e-03, 5.2145e-01,     5.5113e-01, 6.9621e-02, -1.8037e-01, 3.6968e-01, 3.4320e-01,     3.1025e-01, -4.3670e-01, -1.2699e-01, 4.0122e-02, 3.8911e-02,     4.6784e-01, 8.3722e-02, 3.5896e-01, 3.1250e-01, 2.5554e-01],    [-4.1156e-01, -5.9552e-02, -1.4870e-01, 4.3369e-02, 1.7588e-01,     -1.7255e-01, -8.2362e-02, -1.8169e-02, -1.1284e-01, 3.2997e-02,     -1.9642e-01, 6.8670e-02, -1.5620e-01, 3.9276e-02, -1.3103e-01,     8.9772e-02, -3.6906e-01, -1.5121e-01, -1.6676e-01

[2022-07-10T15:47:32.699114+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 27f2ba952a3a438ebd796fbc10b5662e>:
Value: <Storable: 'in_features=30, out_features=3, bias=True'>
[2022-07-10T15:47:32.830819+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: bdc23a1412564d35bc3f2a75a582a8d4>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.5316, -0.1852, 0.3159, 0.2549, 0.3480, -0.2798, 0.4480, -0.1324,     0.3700, 0.4037, 0.3535, 0.3211, 0.4437, 0.4184, 0.2287, -0.3397,     -0.4842, 0.3812, -0.2630, 0.2609, 0.2530, 0.6257, -0.2308, 0.4172,     0.4490, 0.4605, 0.0856, 0.2625, -0.5160, 0.3278],    [ 0.0991, 0.1906, 0.2889, 0.4411, -0.3307, 0.4357, 0.2479, -0.0231,     0.5606, -0.2061, -0.0504, 0.3429, -0.0201, -0.4602, -0.4314, 0.2146,     0.2684, 0.3567, 0.1866, 0.2467, -0.4851, -0.0717, 0.2585, 0.3248,     0.0209, -0.3097, -0.0247, 0.3427, 0.2001, -0.4621],    [-0.5298, -0.3673, -0.2726, -0.3171, -0.3337, -0.3144, -0.2939, 0.0154,     -0.4298, -0.5016, -0.6733, -0.4486,

[2022-07-10T16:01:49.283011+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: ef497f9cbe464cba877b7e609a28e7b6>:
Value: <Storable: [Parameter containing:tensor([[ 0.0178, -0.5114, -0.9144, -0.3233],    [-0.1618, -0.4166, -1.0175, -0.4378],    [-0.3976, -0.1940, -0.2478, -0.5188],    [-0.1380, -0.4091, 0.0014, 0.4134],    [ 0.0227, -0.0133, -0.2915, -1.1410],    [-0.1855, 0.0480, -0.4581, -0.2016],    [-0.0667, -0.7287, -0.0245, -0.1607],    [-0.2686, -0.3616, -0.3187, 0.0046],    [-0.4551, 0.3928, 0.3300, -0.1806],    [ 0.1730, 0.7135, -0.4616, -1.0475],    [ 0.2288, 0.1352, -0.8260, -0.7280],    [-0.1607, -0.1656, -0.7050, -0.2117],    [-0.2105, -0.2670, -0.2767, -0.1688],    [-0.0371, 0.0421, -0.3847, 0.3509],    [-0.3992, -0.1279, -0.5426, -0.7000],    [-0.4986, -0.6482, -0.0982, -0.4327],    [-0.4798, -0.1110, -0.0200, 0.1772],    [-0.3740, -0.3092, -0.7598, -0.7096],    [-0.2254, -0.5938, -0.3903, -0.3451],    [-0.4078, 0.3210, 0.1967, 0.4145]], requires_grad=True), Paramet

[2022-07-10T16:02:09.672210+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 29a1693757cb43b1b99c918f9ee0ca2f>:
Value: <Storable: 'in_features=4, out_features=20, bias=True'>
[2022-07-10T16:02:10.782953+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 9e9a889270fd4c9a9c128b1bce823013>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.0178, -0.5114, -0.9144, -0.3233],    [-0.1618, -0.4166, -1.0175, -0.4378],    [-0.3976, -0.1940, -0.2478, -0.5188],    [-0.1380, -0.4091, 0.0014, 0.4134],    [ 0.0227, -0.0133, -0.2915, -1.1410],    [-0.1855, 0.0480, -0.4581, -0.2016],    [-0.0667, -0.7287, -0.0245, -0.1607],    [-0.2686, -0.3616, -0.3187, 0.0046],    [-0.4551, 0.3928, 0.3300, -0.1806],    [ 0.1730, 0.7135, -0.4616, -1.0475],    [ 0.2288, 0.1352, -0.8260, -0.7280],    [-0.1607, -0.1656, -0.7050, -0.2117],    [-0.2105, -0.2670, -0.2767, -0.1688],    [-0.0371, 0.0421, -0.3847, 0.3509],    [-0.3992, -0.1279, -0.5426, -0.7000],    [-0.4986, -0.6482, -0.0982, -0.4327],    [

[2022-07-10T16:02:14.109907+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 6ea7084730464c618de8b9b27d159b51>:
Value: <Storable: 'in_features=30, out_features=30, bias=True'>
[2022-07-10T16:02:15.226364+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 3455862135324314957c5e9656e0bff8>:
Value: <Storable: OrderedDict([('weight', tensor([[ 2.9269e-01, -1.6785e-01, 4.5387e-01, 2.6658e-01, -3.5345e-02,     5.1731e-01, 5.3217e-01, 5.3964e-01, 4.4317e-01, 5.0218e-01,     -1.6939e-02, 5.1608e-01, -2.6000e-01, -3.0642e-03, 5.2145e-01,     5.5113e-01, 6.9621e-02, -1.8037e-01, 3.6968e-01, 3.4320e-01,     3.1025e-01, -4.3670e-01, -1.2699e-01, 4.0122e-02, 3.8911e-02,     4.6784e-01, 8.3722e-02, 3.5896e-01, 3.1250e-01, 2.5554e-01],    [-4.1156e-01, -5.9552e-02, -1.4870e-01, 4.3369e-02, 1.7588e-01,     -1.7255e-01, -8.2362e-02, -1.8169e-02, -1.1284e-01, 3.2997e-02,     -1.9642e-01, 6.8670e-02, -1.5620e-01, 3.9276e-02, -1.3103e-01,     8.9772e-02, -3.6906e-01, -1.5121e-01, -1.6676e-01

[2022-07-10T16:02:16.316516+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: bdfa801e25cc4d8daa63f01dcbf5f3c0>:
Value: <Storable: 'in_features=30, out_features=3, bias=True'>
[2022-07-10T16:02:16.451377+0300][CRITICAL][logger]][849018] > HANDLER Request <UID: 6494ed903da443bbadde8409a659674b>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.5316, -0.1852, 0.3159, 0.2549, 0.3480, -0.2798, 0.4480, -0.1324,     0.3700, 0.4037, 0.3535, 0.3211, 0.4437, 0.4184, 0.2287, -0.3397,     -0.4842, 0.3812, -0.2630, 0.2609, 0.2530, 0.6257, -0.2308, 0.4172,     0.4490, 0.4605, 0.0856, 0.2625, -0.5160, 0.3278],    [ 0.0991, 0.1906, 0.2889, 0.4411, -0.3307, 0.4357, 0.2479, -0.0231,     0.5606, -0.2061, -0.0504, 0.3429, -0.0201, -0.4602, -0.4314, 0.2146,     0.2684, 0.3567, 0.1866, 0.2467, -0.4851, -0.0717, 0.2585, 0.3248,     0.0209, -0.3097, -0.0247, 0.3427, 0.2001, -0.4621],    [-0.5298, -0.3673, -0.2726, -0.3171, -0.3337, -0.3144, -0.2939, 0.0154,     -0.4298, -0.5016, -0.6733, -0.4486,